---
title: Redfield Issue check "Analytically"
date: 2024-08-08
authors:
  - name: Gerardo Suarez
---

Since there seems to be an issue with the Bloch-Redfield Solver in qutip and my not so good implementation 
of the time dependent redfield, here's a quick sympy check to make sure it's not the solvers. By solving the
equation for the SYK(2) model symbolically. The Hamiltonian is given by

In [1]:
from sympy import *
from sympy.physics.quantum import Dagger
from sympy import I
from collections import defaultdict
import numpy as np
import itertools
from sympy.solvers.ode.systems import dsolve_system

In [2]:
a = symbols('a',real=True,positive=True)

In [3]:
H = diag(-a,a,a,-a)#the times 20 is to rescale time

In [4]:
b= symbols('b0:8', real=True)

Here I define the coupling operator $q$ that make things break for Bloch-Redfield on the SYK model

In [5]:
# coupling operator non trivial like in the shinsey ryu paper. This difference is not dramatic for simpler couplings
Q = Matrix([[0, b[0] - I*b[1], b[2] - I*b[3], 0],
              [b[0] + I*b[1], 0, 0, -b[2] + I*b[3]],
              [b[2] + I*b[3], 0, 0, b[0] - I*b[1]],
              [0, -b[2] - I*b[3], b[0] + I*b[1], 0]])

In [6]:
Dagger(Q) == Q #checking I didn't mess up 

True

In [7]:
def comm(a,b):
    return (a*b-b*a)

I then get the eigenvalues and eigenvectors to obtain the jump operators (this steps are hidden in the pdf)

In [8]:
basis=H.eigenvects() # I probably don't need this step since the Hamiltonian is
# Diagonal
energies=[[i[0]]*i[1] for i in basis]
energies = [x for xs in energies for x in xs]
states = [i[2] for i in basis]
states = [x for xs in states for x in xs ]

In [9]:
def jump_operators(H,evals,all_state, Q):
    """
    A function to obtain the jump operators
    """
    collapse_list = []
    ws = []
    N=len(energies)
    for j in range(N):
        for k in range(j + 1, N):
            Deltajk = evals[k] - evals[j]
            ws.append(Deltajk)
            collapse_list.append(
                (
                    all_state[j]
                    * Dagger(all_state[j])
                    * Q
                    * all_state[k]
                    * Dagger(all_state[k])
                )
            )  # emission
            ws.append(-Deltajk)
            collapse_list.append(
                (
                    all_state[k]
                    * Dagger(all_state[k])
                    * Q
                    * all_state[j]
                    * Dagger(all_state[j])
                )
            )  # absorption
    collapse_list.append(Q - sum(collapse_list,zeros(N)))  # Dephasing
    ws.append(0)
    output = defaultdict(list)
    for k, key in enumerate(ws):
        output[key].append(collapse_list[k])
    eldict = {x: sum(y, zeros(N)) for x, y in output.items()}
    dictrem = {}
    empty = 0*H
    for keys, values in eldict.items():
        if not (values == empty):
            dictrem[keys] = values
    return dictrem

In [10]:
jumps=jump_operators(H,energies,states,Q)

### Jump operator checks

The jump operators must satisfy

$$[H,A(\omega)]=-\omega A(\omega)$$
$$[H,A^{\dagger}(\omega) A(\omega)]=0$$
$$\sum_{w} A(\omega)= A$$

There's a check below hidden in the pdf

In [11]:
comm(H,jumps[2*a])/(-2*a) == jumps[2*a] #checking jump decomposition is fine

True

In [12]:
comm(H,Dagger(jumps[2*a])*jumps[2*a]) == zeros(4)

True

In [13]:
jumps[2*a]+jumps[-2*a] ==Q

True

### Constucting the Differential equations
We now construct the differential equations from the GKLS form of the bloch 
Redfield generator

$$\begin{aligned}
     \dot{\rho_{S}^{I}(t)}{t} &=
    \sum_{\omega,\omega',\alpha,\beta} \gamma_{\beta,\alpha}(\omega,\omega')  \left( S_{\alpha}(\omega')  \rho^{I}_{S}(t) S^{\dagger}_{\beta}(\omega)- \frac{ \{S^{\dagger}_{\beta}(\omega) S_{\alpha}(\omega'),\rho^{I}_{S}(t)\}}{2} \right) \nonumber \\
    &+i \sum_{\omega,\omega',\alpha,\beta} S_{\beta,\alpha}(\omega,\omega') \Big[ \rho^{I}_{S}(t), S^{\dagger}_{\beta}(\omega) S_{\alpha}(\omega')\Big]\end{aligned}$$

I solve in the interaction picture generally, I did the same in my numerics so it should not be an issue (I rotate in the end).
By neglecting Lambshift as in the numerics

$$
     \dot{\rho_{S}^{I}(t)}{t} =
    \sum_{\omega,\omega',\alpha,\beta} \gamma_{\beta,\alpha}(\omega,\omega')  \left( S_{\alpha}(\omega')  \rho^{I}_{S}(t) S^{\dagger}_{\beta}(\omega)- \frac{ \{S^{\dagger}_{\beta}(\omega) S_{\alpha}(\omega'),\rho^{I}_{S}(t)\}}{2} \right)$$


In [14]:
# Simply define the symbolic density operator
rho = symbols('rho_1:17')
rho = Matrix(
    [rho[0: 4],
     rho[4: 8],
     rho[8: 12],
     rho[12:]]).subs(
    rho[-1],
    1 - rho[0] - rho[5] - rho[10])
rho=rho.subs({rho[4]: conjugate(rho[1]), rho[8]: conjugate(rho[2]),
          rho[12]:conjugate(rho[3]),rho[6]:conjugate(rho[9]),
          rho[13]: conjugate(rho[7]), rho[14]:conjugate(rho[11])})
Eq(S('rho'), rho, evaluate=False)


Eq(rho, Matrix([
[           rho_1,            rho_2,             rho_3,                       rho_4],
[conjugate(rho_2),            rho_6, conjugate(rho_10),                       rho_8],
[conjugate(rho_3),           rho_10,            rho_11,                      rho_12],
[conjugate(rho_4), conjugate(rho_8), conjugate(rho_12), -rho_1 - rho_11 - rho_6 + 1]]))

In [15]:
# This bit is simply the GKLS form without the appropiate coefficients
def matrix_form(jumps, combinations,rho):
    matrixform = {}
    for i in combinations:
        matrixform[i] = (
            jumps[i[1]]*rho *Dagger(jumps[i[0]]) -
            (0.5 *
                (Dagger(jumps[i[0]]) * jumps[i[1]]*rho +
                rho*Dagger(jumps[i[0]])* jumps[i[1]])))
    return matrixform

As the sum goes on $(\omega,\omega')$ pairs I construct all combinations

In [16]:
ws = list(jumps.keys())
combinations = list(itertools.product(ws, ws))
combinations

[(2*a, 2*a), (2*a, -2*a), (-2*a, 2*a), (-2*a, -2*a)]

I get the GKLS form of each of those combinations, as a dictionary

In [17]:
matrices=matrix_form(jumps,combinations,rho)

Then I construct the generator by multiplying the appropiate coefficient to each of the GKLS from matrices. Now for the coefficients we have

$$\begin{aligned}
    \Gamma_{\alpha,\beta}(\omega,t) = \int_{0}^{t} ds e^{i \omega s}  \langle B^{\dagger}_{\alpha}(t) B_{\beta}^{\dagger}(t-s) \rangle_{B} \end{aligned}$$

For convenience we also define

$$\begin{aligned}
    \Gamma_{\alpha,\beta}(\omega,\omega',t) = e^{i (\omega'-\omega) t} \int_{0}^{t} ds e^{i \omega s}  \langle B_{\alpha}^{\dagger}(t) B_{\beta}(t-s) \rangle_{B} =e^{i (\omega'-\omega) t} \Gamma_{\alpha,\beta}(\omega,t) \end{aligned}$$

Since I mainly care about bloch-redfield I make $t \xrightarrow{} \infty$ (in the integral) so

$$\begin{aligned}
    \Gamma_{\alpha,\beta}(\omega,\omega',t) = e^{i (\omega'-\omega) t} \int_{0}^{\infty} ds e^{i \omega s}  \langle B(s) B(0) \rangle_{B} =e^{i (\omega'-\omega) t} \Gamma_{\alpha,\beta}(\omega) \end{aligned}$$

Where $\Gamma_{\alpha,\beta}(\omega)$  is the power spectrum

In [18]:
lam,T,w0,t,gam=symbols("lambda T w0 t gamma",real=True)

In [19]:
def power_spectrum(w):
    if w==0:
        return 2 * lam**2 * gam / ( w0**4 * 1/T)
    else:
        return 2 * lam**2 * gam * w / ((w0**2 - w**2)**2 + gam**2 * w**2) * ((1 / (exp(w /T) - 1)) + 1)

In [20]:
def generator(combinations, matrices):
    gen = [
        limit(power_spectrum(s[0]),T,0) * matrices[s]
        for s in combinations]
    return sum(gen, zeros(4))      

In [21]:
gene = generator(combinations,matrices)

Then the generator of the dynamics is given by

In [22]:
Eq(S('G'), gene, evaluate=False)

Eq(G, Matrix([
[                                                                                                                                                                                                                                    4*a*gamma*lambda**2*((b0 + I*b1)*(rho_10*(b2 - I*b3) + rho_6*(b0 - I*b1)) + (b2 + I*b3)*(rho_11*(b2 - I*b3) + (b0 - I*b1)*conjugate(rho_10)))/(4*a**2*gamma**2 + (-4*a**2 + w0**2)**2),                                                                                                                                                             4*a*gamma*lambda**2*(-0.5*(b0 - I*b1)*(rho_2*(b0 + I*b1) + rho_3*(b2 + I*b3)) - 0.5*(-b2 - I*b3)*(rho_2*(-b2 + I*b3) + rho_3*(b0 - I*b1)))/(4*a**2*gamma**2 + (-4*a**2 + w0**2)**2),                                                                                                                                                                         4*a*gamma*lambda**2*(-0.5*(b0 + I*b1)*(rho_2*(-b2 + I*b3) + rho_3*(

Next we simply vectorize the density matrix and construct the system of ODES

In [23]:
mgene = Matrix(list(gene))
rhom=Matrix(list(rho))

In [24]:
rhof = symbols('rho_1:17',cls=Function) # This is necessary to solve the differential equation
to_funcs={i:rhof[k](t) for k,i in enumerate(rhom)} #could have done so from the beginning though

In [25]:
mgene=mgene.subs(to_funcs) #substitute symbols for a function
mrhof=rhom.subs(to_funcs)

In [26]:
eqs = [Eq(mrhof[k].diff(t), mgene[k].simplify()) for k in range(16)]

Let me make a few change of variables, and call the new variables $c_{k}$

In [27]:
c=symbols("c0:10")

In [28]:
c0=mgene[14].simplify()/mrhof[14]

In [29]:
c1=(gam*lam**2)/fraction(mgene[0])[1].simplify()

In [30]:
c2=(b[0]+I*b[1])

In [31]:
c3=(b[2]+I*b[3])

In [32]:
chgnvar={2*c0:2*c[0],c0:c[0],c1:c[1],c2:c[2],conjugate(c2):conjugate(c[2]),
         c3:c[3],conjugate(c3):conjugate(c[3])}

In [33]:
eqs=[i.subs(chgnvar).expand().simplify() for i in eqs]

By substituting these into the differential equation we obtain

With The number of symbols reduced the symbolic computation is feasible. However the default solver with initial conditions yields

In [34]:
sols = dsolve(
    eqs,
    ics={rhof[0] (0): 0, rhof[1] (0): 0, rhof[2] (0): 0, rhof[3] (0): 0,
         rhof[4] (0): 0, rhof[5] (0): 0.5, rhof[6] (0): 0.5 * I, rhof[7]
          (0): 0, rhof[8] (0): 0, rhof[9] (0): -0.5 * I, rhof[10] (0): 0.5,
         rhof[11](0): 0, rhof[12](0): 0, rhof[13](0): 0, rhof[14](0): 0,
         rhof[15](0): 0})

{eval}`sols[4]`

Unfortunately there's a bug in the sympy analytical solver when substituting the initial value conditions to obtain the constants
It's not so bad because it is evidend that the weird term is zero. But I check it with manual substitutions anyway below

:::{warning} 
It's only evident if the solution of the equation is a valid density matrix 
:::
 The initial state considered is

In [35]:
sols = dsolve(
    eqs)

In [36]:
C= symbols("C1:17")

In [37]:
rho0=Matrix([[0]*4,[0,0.5,0.5*I,0],[0,-0.5*I,0.5,0],[0]*4])
rho0

Matrix([
[0,      0,     0, 0],
[0,    0.5, 0.5*I, 0],
[0, -0.5*I,   0.5, 0],
[0,      0,     0, 0]])

{eval}`Eq(S("rho(0)"),rho0,evaluate=False)`

The solution to the system of equations is

Then  by substituting this into the solution we obtain to find the constants we obtain

In [38]:
rho0=list(rho0)
ics=[]
trivial=[]
for k,i in enumerate(sols):
    ic=i.subs(t,0).subs(rhof[k](0),rho0[k]).simplify()
    if ic.rhs==0:
        trivial.append(ic.lhs)
    else:
        ics.append(ic)
    display(ic)

Eq((1.0*C1*c0*c3 - C2*c0*conjugate(c2) + 2.0*C3*a*c1*c3*(c2*conjugate(c2) + c3*conjugate(c3)) + 2.0*C4*a*c1*c2*(c2*conjugate(c2) + c3*conjugate(c3)))/(c0*c3), 0)

Eq((-C5*a*c1*(c2*conjugate(c2) + c3*conjugate(c3))*conjugate(c2) - 0.25*C6*c3*(2.0*a*c1*(c2*conjugate(c2) + c3*conjugate(c3)) + c0) + 0.25*C7*(2.0*a*c1*(c2*conjugate(c2) + c3*conjugate(c3)) + c0)*conjugate(c2))/(a*c1*c3*(c2*conjugate(c2) + c3*conjugate(c3))), 0)

Eq((-C8*a*c1*(c2*conjugate(c2) + c3*conjugate(c3))*conjugate(c2) + (C6*c2 + C7*conjugate(c3))*(0.5*a*c1*(c2*conjugate(c2) + c3*conjugate(c3)) + 0.25*c0))/(a*c1*c3*(c2*conjugate(c2) + c3*conjugate(c3))), 0)

Eq((-C10*c0*conjugate(c2) + 1.0*C9*c0*c3 + 2.0*a*c1*(C3*conjugate(c2) - C4*conjugate(c3))*(c2*conjugate(c2) + c3*conjugate(c3)))/(c0*c3), 0)

Eq(1.0*C11 - 4.0*C5*a*c1*c2*(c2*conjugate(c2) + c3*conjugate(c3))/(c3*(2.0*a*c1*(c2*conjugate(c2) + c3*conjugate(c3)) + c0)) + C7*c2/c3 - 4.0*C8*a*c1*(c2*conjugate(c2) + c3*conjugate(c3))/(2.0*a*c1*(c2*conjugate(c2) + c3*conjugate(c3)) + c0), 0)

...

Eq(1.0*C14, -1.0*C6)

Eq(1.0*C15, -1.0*C2)

Eq(C5, 0)

Eq(C8, 0)

Eq(1.0*C10, -1.0*C16)

In [39]:
constants_zeros={i:0 for i in trivial}

In [40]:
minus={C[0]:-C[1],C[4]:-C[5],C[14]:-C[10],C[15]:-C[9]}

In [41]:
c10=solve(ics[2],C[9])[0]

IndexError: list index out of range

In [ ]:
c11=solve(ics[3].subs(C[9],c10),C[10])[0]

In [ ]:
c6=solve(ics[4].subs(C[9],c10).subs(C[10],c11),C[5])[0]

In [ ]:
c02=solve(ics[5].subs(C[9],c10).subs(C[10],c11).subs(C[5],c6),C[1])[0].simplify()

then

In [ ]:
c06=c6.subs(C[1],c02).simplify()

In [ ]:
c011=c11.subs(C[1],c02).subs(C[5],c06).simplify()

In [ ]:
c010=c10.subs(C[1],c02).subs(C[5],c06).subs(C[10],c011).simplify()

In [ ]:
simplified={C[9]:c010,C[1]:c02,C[10]:c011,C[5]:c06}

After solving the system of equations above we obtain

In [ ]:
ic2=[]
for i in sols:
    j=i.simplify().subs(constants_zeros).subs(minus).subs(simplified)
    ic2.append(j)
    display(j)

Though simplified there are still some Free constants, which we can get rid off, since by
substituting the ones we found the piecewise conditions are now fully determined such that once
again using the initial conditions

so finally

In [ ]:
ic2=[]
for i in sols:
    j=i.simplify().subs(constants_zeros).subs(minus).subs(simplified)
    ic2.append(j)
    display(j)

In [ ]:
inverted = {value: key for key, value in chgnvar.items()}

In [ ]:
newsols=[i.subs(inverted).simplify() for i in ic2]

By reverting the change of variables. The analytical answer of the Bloch redfield equation is given by

In [ ]:
ans=Matrix(np.array([i.rhs for i in newsols]).reshape(4,4))

We can then substitute the numerical values for example for the case we explored above

In [ ]:
num_values = {w0: 4.799892154530513, lam: 2.157529680721121, gam: 2.3414108070880553, a: 2.182,b[0]:0.53,b[1]:0.707,b[2]:0.442,b[3]:0.471}

In [ ]:
anss=ans.subs(num_values).evalf()

{eval}`Eq(S("rho(t)"),anss,evaluate=False)`

Then we may evaluate for long times

In [ ]:
def roundMatrix(m, n): 
    return Matrix(
    [[round(m[x, y], n) for y in range(m.shape[1])] for x in range(m.shape[0])])

{eval}`Eq(S("rho(50)"),roundMatrix(ans.subs(num_values).subs(t,150).evalf(),18),evaluate=False)`

I believe I was careful enough to use the same convention used in the other equations (Pseudomodes, Cumulant and redfield ) but currently reviewing the derivations to make sure there's no inconsistencies. The derivations in question are in https://master--gsuarezthesis.netlify.app/redfield . I do think it is now safe to assume that BR/Redfield breaksdown and that it is not a bug in the code, so maybe we can write a paper on redfield breaking down, cumulant/global being good once we figure out why it happens🤓. Though it seems to be about the coupling and not the degeneracies

:::{tip} About Pictures
Technically the above matrix is not correct as it is in the interaction picture and not the Schrodinger picture. In this case it does not make a difference, however, let us do the rotation 
:::

In [ ]:
U=exp(I*H*t)

{eval}`Eq(S("U"),U,evaluate=False)`

{eval}`Eq(S("U")*S("rho")*Dagger(S("U")),U*rho*Dagger(U),evaluate=False)`


In [ ]:
rhoss=U*roundMatrix(ans.subs(num_values).subs(t,150).evalf(),18)*Dagger(U)

{eval}`Eq(S("rho(50)"),rhoss,evaluate=False)`


## RC picture of the Hamiltonian

For the RC I simply follow one of your papers https://arxiv.org/pdf/1511.05181

So If I didn't misunderstand it then

$$\Omega=w_{0}$$
$$\lambda_{rc}=\sqrt{\frac{\pi}{2 w_{0}}}\lambda$$


Not actually sure if $\pi$ should be there, but does not seem to be relevant for the question we are asking

In [ ]:
def ladder(i,j):
    if i - j == 1:
        return sqrt(i)
    else:
        return 0
def destroy(n):
    return Dagger(Matrix(n,n,ladder))


Not sure how many levels to take here, but let us guess 15 is enough

In [ ]:
levels=15
arc=destroy(levels)
arcd=Dagger(arc)

Then I construct the RC Hamiltonian

In [ ]:
from sympy.physics.quantum import TensorProduct

In [ ]:
HRC = TensorProduct(H, eye(levels))+sqrt(pi/(2*w0))*lam*TensorProduct(Q,eye(levels))*(TensorProduct(eye(4),arc)+TensorProduct(eye(4),arcd))+w0*TensorProduct(eye(4),arcd)*TensorProduct(eye(4),arc)

In [ ]:
NHRC=HRC.subs(num_values).evalf()

In [ ]:
ans=np.array(NHRC.tolist()).astype(np.complex128)

In [ ]:
eigenvalues, eigenvectors = np.linalg.eig(ans)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(np.real(eigenvalues),np.round(np.imag(eigenvalues),10),s=5)
plt.title("Eigenvalues on the real line")
plt.show()

Propably I shopuld have done the partial trace so

In [ ]:
import qutip as qt

In [ ]:
Hq=qt.Qobj(H.subs(num_values).tolist())
aq=qt.destroy(levels)
aqd=qt.create(levels)
qHRC = qt.tensor(Hq, qt.qeye(levels))+np.sqrt(np.pi/(2*num_values[w0]))*num_values[lam]*qt.tensor(qt.Qobj(Q.subs(num_values)), qt.qeye(levels))*(qt.tensor(qt.qeye(4),aq)+qt.tensor(qt.qeye(4),aqd))+num_values[w0]*qt.tensor(qt.qeye(4),aqd)*qt.tensor(qt.qeye(4),aq)

In [ ]:
Hq

In [ ]:
qHRC.ptrace(0)

Still degenerate

Let us try the other coupling which does not break bloch redfield

In [ ]:
Hq=qt.Qobj(H.subs(num_values).tolist())
aq=qt.destroy(levels)
aqd=qt.create(levels)
qHRC = qt.tensor(Hq, qt.qeye(levels))+np.sqrt(np.pi/(2*num_values[w0]))*num_values[lam]*qt.tensor(qt.Qobj(re(Q).subs(num_values)), qt.qeye(levels))*(qt.tensor(qt.qeye(4),aq)+qt.tensor(qt.qeye(4),aqd))+num_values[w0]*qt.tensor(qt.qeye(4),aqd)*qt.tensor(qt.qeye(4),aq)

In [ ]:
qHRC.ptrace(0)

In [ ]:
ans = qHRC.full().astype(np.complex128)

In [ ]:
eigenvalues, eigenvectors = np.linalg.eig(ans)

In [ ]:
plt.scatter(np.real(eigenvalues), np.round(np.imag(eigenvalues), 10), s=5)
plt.title("Eigenvalues on the real line")
plt.show()

There doesn't seem to be much change in the Hamiltonian if any